Імпортуємо потрібну бібліотеку.

In [ ]:
import pandas as pd 

In [ ]:
def getDataset(path):
    return pd.read_csv(path, sep=',',encoding='cp1252')

dataset = getDataset('FullDataset.csv')

dataset = dataset[dataset["ID"]<50000]
dataset.info()
dataset.head(10)

def isMatch(str,lst):
    for l in lst:
        if(l == str):
            return 1
    return 0
lst1 = ["Cross Country Skiing", "Sailing", "Athletics", "Alpine Skiing", "Luge", "Rowing", "Swimming", "Canoeing", "Cycling", "Freestyle Skiing"]
lst2 = ["Speed Skating", "Weightlifting", "Bobsleigh", "Archery", "Shooting", "Trampolining", "Snowboarding", "Ski Jumping", "Short Track Speed Skating", "Skeleton"]
lst3 = ["Gymnastics", "Diving", "Figure Skating", "Synchronized Swimming", "Rhythmic Gymnastics", "Curling", "Golf"]
lst4 = ["Judo", "Wrestling", "Taekwondo", "Boxing", "Fencing"]
lst5 = ["Biathlon", "Modern Pentathlon", "Nordic Combined", "Triathlon"]
lst6 = ["Basketball","Ice Hockey", "Badminton", "Football", "Equestrianism", "Handball", "Water Polo", "Tennis", "Hockey", "Softball", "Volleyball", "Table Tennis", "Baseball", "Rugby Sevens", "Beach Volleyball"]

for i in range(len(dataset)) :
    if(isMatch(dataset.iloc[i]["Sport"], lst1) == 1):
        dataset.at[i,"Sport"] = 'Cyclic'
    elif(isMatch(dataset.iloc[i]["Sport"], lst2) == 1):
        dataset.at[i,"Sport"] = 'Speed'
    elif(isMatch(dataset.iloc[i]["Sport"], lst3) == 1):
        dataset.at[i,"Sport"] = 'Coordination'
    elif(isMatch(dataset.iloc[i]["Sport"], lst4) == 1):
        dataset.at[i,"Sport"] = 'Fight'
    elif(isMatch(dataset.iloc[i]["Sport"], lst5) == 1):
        dataset.at[i,"Sport"] = 'All-around'
    else:
        dataset.at[i,"Sport"] = 'Games'


dataset.to_csv("check1.csv", sep=',', encoding='utf-8')

Приберемо непотрібні для нашого аналізу стовпчики.

In [ ]:
dataset = dataset.drop("Unnamed: 0", axis = 1)
dataset = dataset.drop("Code", axis = 1)
dataset = dataset.drop("Year", axis = 1)
dataset = dataset.drop("Team", axis = 1)
dataset = dataset.drop("ID", axis = 1)
dataset.info()
dataset.head(10)

Змінимо значення поля з медалями як булеве чи здобув спортсмен медаль.

In [ ]:
dataset["Medal"].notnull().astype('int')
dataset["Medal"].fillna(0,inplace = True)

dataset["Medal"]=dataset["Medal"].replace(regex=["Gold"],value='1')
dataset["Medal"]=dataset["Medal"].replace(regex=["Bronze"],value='1')
dataset["Medal"]=dataset["Medal"].replace(regex=["Silver"],value='1')
dataset.to_csv("check1.csv", sep=',', encoding='utf-8')

dataset.info()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(x="Medal", data=dataset, palette = "Purples")
plt.show()

Ми готові працювати з даними. Тепер давайте розподілимо дані на тренувальні та дані для тесту.

In [ ]:
from sklearn.model_selection import train_test_split
trainingData, testingData  = train_test_split(dataset,test_size=0.30,random_state = 1)

features = pd.concat([trainingData,testingData]).reset_index(drop = True)
features = pd.get_dummies(features)
features = features.drop("Medal_0", axis = 1)
features.rename(columns = {'Medal_1':'Medal'}, inplace = True)
trainingData = features.iloc[:trainingData.shape[0],:]
testingData = features.iloc[trainingData.shape[0]:,:]
print(trainingData.head(10))


In [ ]:
sns.heatmap(dataset.corr(),annot=True,lw=1,cmap = "Purples")

Тепер визначимо змінні X та Y для тренувальних та тестувальних даних.

In [ ]:
trainingX  = trainingData.drop(columns='Medal')
trainingY = trainingData['Medal']

testingX  = testingData.drop(columns='Medal')
testingY = testingData['Medal']

Додамо необхідні бібліотеки для побудови моделей, побудуємо їх та перевіримо на тестових даних. Спочатку класифікатор Decision Tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

decisionTree = DecisionTreeClassifier(max_depth=15,random_state=1)
decisionTree.fit(trainingX,trainingY)

plt.figure(figsize=(30,30))  
tree.plot_tree(decisionTree)
plt.show()

arr = decisionTree.predict(testingX)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm,title = "Confusion matrix",cmap = plt.cm.Purples):
    plt.figure()
    print(cm)   
    plt.imshow(cm,interpolation="nearest",cmap=cmap)
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.show()

cm = confusion_matrix(testingY,arr)
plot_confusion_matrix(cm)

In [ ]:
print('Score of Decision Tree:',decisionTree.score(testingX,testingY)) 

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def plot_roc_curve(fper, tper):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
probs = decisionTree.predict_proba(testingX)  
probs = probs[:, 1]  
fper, tper, thresholds = roc_curve(testingY, probs) 

plot_roc_curve(fper, tper)
auc = roc_auc_score(testingY, probs)
print("AUC score:",auc)

Тепер класифікатор Random Forest. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(max_depth=15)
randomForest.fit(trainingX,trainingY)

arr = randomForest.predict(testingX)

In [ ]:
cm = confusion_matrix(testingY,arr)
plot_confusion_matrix(cm)

In [ ]:
print('Score of Random Forest:',randomForest.score(testingX,testingY)) 

In [ ]:
probs = randomForest.predict_proba(testingX)  
probs = probs[:, 1]  
fper, tper, thresholds = roc_curve(testingY, probs) 

plot_roc_curve(fper, tper)
auc = roc_auc_score(testingY, probs)
print("AUC score:",auc)